## 1. Dependencies

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt # Plotting library
import numpy as np # Algebra library
# import tensorflow as tf # Tensorflow
import tensorflow.compat.v1 as tf # Tensorflow
import os # Folder management library
import pandas as pd # Essentially puts data in spreadsheeet
#from PIL import Image # Image processing library
import scipy.misc # General stats functions
import random # Pseudo random number generator
from sklearn.model_selection import train_test_split # To split data into train, test, validation
import pickle # Serializing module.
from matplotlib.image import imread # Plotting images

# New libraries ------------------------------------------------------------
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.initializers import glorot_uniform
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
import tensorflow as tf2

In [ ]:
# Print tensorflow version
print('Tensorflow version:', tf.__version__)

strategy = tf2.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

### Available CPUs and GPUs on Machine
Just shows what devices are available.

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## 2. Load Data for Training, Validation, and Testing 
Run this cell if you will be training the architecture.

In [ ]:
# Read data from disk. X contains images, and y contains labels.
X_unpickle = open('X_new.pickle', 'rb')
y_unpickle = open('y_new.pickle', 'rb')

# load the unpickle object into a variable
X = pickle.load(X_unpickle)
y = pickle.load(y_unpickle)

# Split data into train, validation, test
# Stratify with respect to y so that there is an even number of 1s and 0s in each set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=50/350, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=50/300, random_state=42, stratify=y_train)
# Convert lists to numpy arrays
X_train, X_test, X_val = np.array(X_train).astype("float32"), np.array(X_test).astype("float32"), np.array(X_val).astype("float32")
y_train, y_test, y_val = np.array(y_train).astype("float32"), np.array(y_test).astype("float32"), np.array(y_val).astype("float32")

print('Training Set (No. imgs, length, width) and Labels (No. imgs):', X_train.shape, y_train.shape)
print('Validation Set (No. imgs, length, width) and Labels (No. imgs):', X_val.shape, y_val.shape)
print('Test Set (No. imgs, length, width) and Labels (No. imgs):', X_test.shape, y_test.shape)

### Visualize Data

In [ ]:
plt.figure(figsize=(18, 11))
for i in range(1, 6):
    plt.subplot(1, 5, i)
    sample_image = X_train[i]
    plt.imshow(sample_image, cmap='gray')
    plt.title(y_train[i])
    plt.axis("off")
plt.show()

## Model Architecture

In [ ]:
with strategy.scope():


    # FUNCTIONS ------------------------------------------------------------------------------------------------------------
    class predictions(tf.keras.layers.Layer):

        def __init__(self, caps_Lminus1, dims_Lminus1, caps_L, dims_L, name=None):
            super(predictions, self).__init__()
            init_sigma = 0.1
            W_init = tf.random_normal(shape=(1, caps_Lminus1, caps_L, dims_L, dims_Lminus1),
                                      stddev=init_sigma, dtype=tf.float32, name="W_init")            
            self.W = tf.Variable(W_init, name="W")
            

        def call(self, inputs, **kwargs):
            output_Lminus1, batch_size, weight_sharing, grid_cells_Lminus1, caps_L = inputs

            if tf.get_static_value(weight_sharing) == False:
                output_Lminus1 = tf.squeeze(output_Lminus1, axis=[1, 4], name="output_Lminus1")
                
            # Tile W for the batch
            W_tiled = tf.tile(self.W, [batch_size, grid_cells_Lminus1, 1, 1, 1], name="W_tiled")
            # -1 means add another dimension to the end of tensor
            output_expanded_Lminus1 = tf.expand_dims(output_Lminus1, -1, name="output_expanded_Lminus1")
            # 2 means add another dimension to the 2th position of tensor
            output_tile_Lminus1 = tf.expand_dims(output_expanded_Lminus1, 2, name="output_tile_Lminus1")
            # 1 in shape argument means keep dimensions from caps1_output_tile
            output_tiled_Lminus1 = tf.tile(output_tile_Lminus1, [1, 1, caps_L, 1, 1], name="output_tiled_Lminus1")
            # Get prediction tensor (3rd array) by using t.matmul
            predicted_L = tf.matmul(W_tiled, output_tiled_Lminus1, name="predicted_L")

            return predicted_L


    def routing(caps_Lminus1, caps_L, batch_size, iterations, predicted_L, name=None):

        with tf.name_scope(name, default_name="routing"):
            # initialize the raw routing weights to zero
            # The two extra 1s is so that raw_weights and weighted_predictions have the same size
            raw_weights = tf.zeros([batch_size, caps_Lminus1, caps_L, 1, 1], dtype=tf.dtypes.float32, name="raw_weights")

            for r in range(0, iterations):

                # c_i = softmax(b_i)
                routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")
                # weighted sum of all the predicted output vectors for each second-layer capsule
                weighted_predictions = tf.multiply(routing_weights, predicted_L, name="weighted_predictions")
                weighted_sum = tf.reduce_sum(weighted_predictions, axis=(1), keepdims=True, name="weighted_sum")

                # Squash weighted_sum (s_j) to get output for 2nd capsules (v_j)
                output_L = squash(weighted_sum, axis=-2, name="output_L")

                # Make caps2_output same size as predicted_caps
                output_tiled_L = tf.tile(output_L, 
                                             [1, caps_Lminus1, 1, 1, 1], 
                                             name="output_tiled_L")
                # the 1 in the shape argument in tf.tile means keep dimension from caps2_output.

                # Dot Product
                agreement = tf.matmul(predicted_L, output_tiled_L, transpose_a=True, name="agreement")
                # Update raw_weights
                raw_weights = tf.add(raw_weights, agreement, name="raw_weights")

            return output_L, routing_weights

    # safe_norm taken from Aurélien Geron
    def squash(s, axis=-1, epsilon=1e-7, name=None):

        with tf.name_scope(name, default_name="squash"):
            squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                         keepdims=True)
            safe_norm = tf.sqrt(squared_norm + epsilon)
            squash_factor = squared_norm / (1. + squared_norm)
            unit_vector = s / safe_norm
            return squash_factor * unit_vector


    # safe_norm taken from Aurélien Geron
    def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):

        with tf.name_scope(name, default_name="safe_norm"):
            squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                         keep_dims=keep_dims)
            return tf.sqrt(squared_norm + epsilon)
        
    # THE CAPSULE NETWORK ARCHITECTURE ----------------------------------------------------------------------------------            
    #input ---------------------------------------------------------------------
    X = layers.Input(shape=(None, None, 1), dtype=tf.float32, name="X") 
    y = layers.Input(shape=[None], dtype=tf.float32, name="y")
    
    # Conv Layer ---------------------------------------------------------------
    conv1 = layers.Conv2D(filters=16, kernel_size=9, strides=(3,3), 
                          padding="valid", activation='relu')(X)
    
    # Convcaps Layer -----------------------------------------------------------
    # No. of dimenions a single capsule contains.
    convcaps_dims = 8 
    # No. of capsules per grid cell.
    convcaps_caps_types = 32 
    convcaps = layers.Conv2D(filters=convcaps_caps_types*convcaps_dims, kernel_size=5, strides=(2,2), 
                             padding="valid", activation='relu')(conv1)
    # Length and width of convaps layer.
    convcaps_grid_length = tf.shape(convcaps)[1]   
    convcaps_grid_width = tf.shape(convcaps)[2] 
    convcaps_grid_cells = convcaps_grid_length * convcaps_grid_width
    # Total No. capsules in convcaps layer.
    convcaps_caps = convcaps_caps_types * convcaps_grid_cells
    # Reshape convcaps so that it is easier to work with.
    convcaps_reshape = tf.reshape(convcaps, [-1, convcaps_caps, convcaps_dims], name="convcaps_reshape")
    # Squash convcaps with squashing function.
    convcaps_output = squash(convcaps_reshape, name="convcaps_output")
    

    
    # First Capsule Layer-------------------------------------------------------
    # The batch size
    batch_size = tf.shape(X)[0]
    caps1_caps = 24
    caps1_dims = 12
    caps1_predictions = predictions(convcaps_caps_types, convcaps_dims, 
                                    caps1_caps, caps1_dims, "caps1_predictions")([convcaps_output, batch_size, True, convcaps_grid_cells, caps1_caps])
    caps1_output, routing1 = routing(convcaps_caps, caps1_caps, 
                                      batch_size, 9, caps1_predictions, "routing1")
    
    # Second Capsule Layer ------------------------------------------------------
    caps2_caps = 8
    caps2_dims = 16
    caps2_predictions = predictions(caps1_caps, caps1_dims, 
                                    caps2_caps, caps2_dims, "caps2_predictions")([caps1_output, batch_size, False, 1, caps2_caps])
    caps2_output, routing2 = routing(caps1_caps, caps2_caps, batch_size, 9, caps2_predictions, "routing2")


    # Third Capsule Layer --------------------------------------------------------
    caps3_caps = 1
    caps3_dims = 16

    caps3_predictions = predictions(caps2_caps, caps2_dims, 
                                    caps3_caps, caps3_dims, "caps3_predictions")([caps2_output,  batch_size, False, 1, caps3_caps])

    caps3_output, routing3 = routing(caps2_caps, caps3_caps, batch_size, 9, caps3_predictions, "routing3")
    
    
    # Capsule Accuracy ----------------------------------------------------------
    # Lengths of capsules represent probability that entity exists.
    lengths = safe_norm(caps3_output, axis=-2, name="lengths")
    # Round lengths to determine predicted class.
    lengths_rounded = tf.round(lengths, name="lengths_rounded")
    # Remove dimensions of size 1 from lengths_rounded
    y_predictions = tf.squeeze(lengths_rounded, axis=[1,2,3], name="y_predictions")

    # If y matches y_predictions than 1 otherwise 0.
    correct = tf.equal(y, y_predictions, name="correct")
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
    
    # Custom loss----------------------------------------------------------------
    def my_loss_fn(y_true, y_pred):
        m_plus = 0.9
        m_minus = 0.1
        lambda_ = 0.5

        T = tf.reshape(y_true, shape=[-1, 1], name="T")
        # Compute norm of each capsule in digitcaps
        caps3_output_norm = safe_norm(y_pred, axis=-2, keep_dims=True, name="caps3_output_norm")

        present_error_raw = tf.square(tf.maximum(0., m_plus - caps3_output_norm), name="present_error_raw")
        present_error = tf.reshape(present_error_raw, shape=(-1, caps3_caps), name="present_error")
        # -1 tells reshape to calculate the size of this dimension. 

        absent_error_raw = tf.square(tf.maximum(0., caps3_output_norm - m_minus), name="absent_error_raw")
        absent_error = tf.reshape(absent_error_raw, shape=(-1, caps3_caps), name="absent_error")
        # -1 tells reshape to calculate the size of this dimension. 

        # Compute Margin Loss
        L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error, name="L")
        loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="loss")
        tf.summary.scalar('loss', loss)

        return loss


    
    # Optimizer ---------------------------------------------------------------
    model = Model(X, caps3_output)
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss=my_loss_fn)
    
    
    # Callback -----------------------------------------------------------------------
    val_temp_path = './val_temp1.pickle'
    if os.path.isfile(val_temp_path):
        val_temp = open(val_temp_path, 'rb')
        val_temp = pickle.load(val_temp)
    else:
        val_temp = 10

    class custom(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            global val_temp
            val = logs.get('val_loss')
            if val < val_temp:
                val_temp = val
                print(' ')
                print(val_temp, "SAVING")

                with open(val_temp_path, 'wb') as handle:
                    pickle.dump(val_temp, handle, protocol=pickle.HIGHEST_PROTOCOL)

                model.save_weights('./new_weights.h5')
    

    # Load weights --------------------------------------------------------------------------
    model.load_weights('./model/my_capsule_network.h5')    
    

## 4. Train the Architecture
Run the below cell if you would like to train the architecture.

In [ ]:
model.fit(X_train, y_train, batch_size=5, epochs=1000, validation_data=(X_val, y_val), callbacks=[custom()])

### Evaluate on Test Set

In [ ]:
results = model.evaluate(X_test, y_test, batch_size=10)
results

### Total Number of Trainable Weights in Model

In [ ]:
model.summary()